In [1]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
os.chdir("/data/private/pdutta/DNABERT_data/Core_promoters/Core_promoter_regions")

In [3]:
file_extension = '.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

In [4]:
all_filenames

['chr7_core_promoter_sequence.csv',
 'chr13_core_promoter_sequence.csv',
 'chr8_core_promoter_sequence.csv',
 'chr16_core_promoter_sequence.csv',
 'chr12_core_promoter_sequence.csv',
 'chr1_core_promoter_sequence.csv',
 'chr22_core_promoter_sequence.csv',
 'chr9_core_promoter_sequence.csv',
 'chrY_core_promoter_sequence.csv',
 'chr11_core_promoter_sequence.csv',
 'chr4_core_promoter_sequence.csv',
 'chr6_core_promoter_sequence.csv',
 'chr5_core_promoter_sequence.csv',
 'chr14_core_promoter_sequence.csv',
 'chr20_core_promoter_sequence.csv',
 'chrX_core_promoter_sequence.csv',
 'chr10_core_promoter_sequence.csv',
 'chr21_core_promoter_sequence.csv',
 'chr2_core_promoter_sequence.csv',
 'chr3_core_promoter_sequence.csv',
 'chr19_core_promoter_sequence.csv',
 'chr17_core_promoter_sequence.csv',
 'chr18_core_promoter_sequence.csv',
 'chr15_core_promoter_sequence.csv']

In [5]:
dfs = []
for f in all_filenames:
    print(f.split("_")[0])
    df = pd.read_csv(f, sep= ',')
    df['Chr'] = f.split("_")[0]
    dfs.append(df)

chr7
chr13
chr8
chr16
chr12
chr1
chr22
chr9
chrY
chr11
chr4
chr6
chr5
chr14
chr20
chrX
chr10
chr21
chr2
chr3
chr19
chr17
chr18
chr15


In [6]:
combined_csv_data = pd.concat(dfs)
combined_csv_data.drop_duplicates().reset_index(drop=True)

,gene_id,transcript_id,strand,exon_id,transcript_type,TSS,sequence,Chr
0,ENSG00000004059.11,ENST00000000233.10,+,ENSE00001872691.2,protein_coding,127588411,cagcgacgcgcggaggcggcggcggAGCCTCCTCCTGCTGCTGCTG...,chr7
1,ENSG00000001626.16,ENST00000003084.11,+,ENSE00001343851.2,protein_coding,117480025,TCAGAGAAAAAGGGTTGAGCGGCAGGCACCCAGAGTAGTAGGTCTT...,chr7
2,ENSG00000001630.17,ENST00000003100.13,-,ENSE00001216550.10,protein_coding,92134477,GCGCGCGTGGGACCCGAGGGGTGGGGCTGGGTTTAGTAGGAGACCT...,chr7
3,ENSG00000002933.9,ENST00000004103.8,+,ENSE00001827596.2,protein_coding,150800769,tcccctccccgcctccccctccAACTCTCAACCCACTTCTCCAGCC...,chr7
4,ENSG00000004799.8,ENST00000005178.6,-,ENSE00001133344.6,protein_coding,95596516,ACAAATAAAACCTCGGGCGGCGGCGGCTGGTGGGAAGACTTGAACT...,chr7
...,...,...,...,...,...,...,...,...
236970,ENSG00000103876.14,ENST00000684569.1,+,ENSE00003921718.1,processed_transcript,80153010,GCCTGACCACAGCGGCCGAGTTCAGTCCTGCTCTCCGCACGCCACC...,chr15
236971,ENSG00000213614.11,ENST00000684602.1,-,ENSE00002620879.1,nonsense_mediated_decay,72375989,GACCAGGGTCTCACGTGGCCAGCCCCCTCCGAGAGGGGAGACCAGC...,chr15
236972,ENSG00000213614.11,ENST00000684667.1,-,ENSE00002584031.1,nonsense_mediated_decay,72375972,GCCAGCCCCCTCCGAGAGGGGAGACCAGCGGGCCATGACAAGCTCC...,chr15
236973,ENSG00000104133.16,ENST00000684676.1,-,ENSE00003922470.1,nonsense_mediated_decay,44663688,CGCTACGTACACCGCGCATGCCCACGACGCAGTCAGGTTCCGGCGA...,chr15


In [7]:
len(combined_csv_data['sequence'].iloc[0])

70

In [8]:
df_bed = combined_csv_data[['Chr','gene_id', 'transcript_id',	'exon_id', 'strand', 'TSS']]
# Create 'start' and 'end' columns based on 'strand' and 'position'
df_bed['start'] = df_bed.apply(lambda row: row['TSS'] - 45 if row['strand'] == '+' else row['TSS'] - 44, axis=1)
df_bed['end'] = df_bed.apply(lambda row: row['TSS'] + 44 if row['strand'] == '+' else row['TSS'] +45 , axis=1)
df_bed

/tmp/ipykernel_1990249/2930344134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bed['start'] = df_bed.apply(lambda row: row['TSS'] - 45 if row['strand'] == '+' else row['TSS'] - 44, axis=1)


,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr7,ENSG00000004059.11,ENST00000000233.10,ENSE00001872691.2,+,127588411,127588366,127588455
1,chr7,ENSG00000001626.16,ENST00000003084.11,ENSE00001343851.2,+,117480025,117479980,117480069
2,chr7,ENSG00000001630.17,ENST00000003100.13,ENSE00001216550.10,-,92134477,92134433,92134522
3,chr7,ENSG00000002933.9,ENST00000004103.8,ENSE00001827596.2,+,150800769,150800724,150800813
4,chr7,ENSG00000004799.8,ENST00000005178.6,ENSE00001133344.6,-,95596516,95596472,95596561
...,...,...,...,...,...,...,...,...
8684,chr15,ENSG00000103876.14,ENST00000684569.1,ENSE00003921718.1,+,80153010,80152965,80153054
8685,chr15,ENSG00000213614.11,ENST00000684602.1,ENSE00002620879.1,-,72375989,72375945,72376034
8686,chr15,ENSG00000213614.11,ENST00000684667.1,ENSE00002584031.1,-,72375972,72375928,72376017
8687,chr15,ENSG00000104133.16,ENST00000684676.1,ENSE00003922470.1,-,44663688,44663644,44663733


In [9]:
df_bed.to_csv("/data/private/pdutta/DNABERT_data/Core_promoters/Core_Prom_bed.tsv", sep="\t", index=False)

In [6]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [7]:
df_pos=pd.DataFrame()
df_pos['Sequence'] = combined_csv_data['sequence'].str.upper().apply(seq2kmer, args=(6,))

In [8]:
df_pos

,Sequence
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...
...,...
4320,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...
4321,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...
4322,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...
4323,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...


In [9]:
df_pos['Label']=1

In [10]:
df_pos = df_pos.reset_index(drop=True)
df_pos

,Sequence,Label
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...,1
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...,1
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...,1
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...,1
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...,1
...,...,...
236970,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...,1
236971,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1
236972,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...,1
236973,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1


In [12]:
df_pos.to_csv("/data/projects/DNABERT_data/Core_promoters/Core_promoter_regions/positive_set.tsv", sep="\t", index=False)